In [1]:
using LibPQ, DataFrames, DataStreams

In [ ]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    LibPQ.execute(conn, """
        SET work_mem TO '1GB';
    """);
    
    st = LibPQ.prepare(conn, """
        SELECT * FROM ml_small LIMIT \$1 OFFSET \$2;
    """);
    
    for i in 0:1008
        res = LibPQ.execute(st, [1, i])
    end
end

In [ ]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    LibPQ.execute(conn, """
        SET work_mem TO '1GB';
    """);
    
    for i in 0:1008
        LibPQ.execute(conn, """
            SELECT * FROM ml_small LIMIT \$1 OFFSET \$2;
        """, [1, i]);
    end
end

In [ ]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    LibPQ.execute(conn, """
        SET work_mem TO '1GB';
    """);
    
    LibPQ.execute(conn, """
            SELECT * FROM ml_small LIMIT \$1 OFFSET \$2;
        """, [100836, 0]);
end

In [ ]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    LibPQ.execute(conn, """
        SET work_mem TO '1GB';
    """);
    
    LibPQ.execute(conn, """
            SELECT * FROM ml_small;
        """);
end

In [ ]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    res = LibPQ.execute(conn, """
        WITH data AS (
            SELECT
                "userId",
                count(*)
            FROM ml_small
            GROUP BY  "userId"
            ORDER BY "userId"
        )
        SELECT
            "userId",
            sum(count) OVER (
                ORDER BY "userId" ASC
                ROWS UNBOUNDED PRECEDING
            )::INT8 AS cumsum
        FROM data;
        """)
    
    LibPQ.fetch!(DataFrame, res);
end

In [ ]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    res = LibPQ.execute(conn, """
        WITH a AS (
            SELECT
                "userId",
                trunc(
                    cume_dist() OVER (
                        ORDER BY "userId"
                    ) * (\$1 -  0.00001)
                )::int4 + 1 AS chunk
            FROM ml_small
            ORDER BY "userId"
        )
        
        SELECT
            min("userId"),
            max("userId"),
            count("userId"),
            chunk 
        FROM a
        GROUP BY chunk
        ORDER BY chunk;
    """, [10]);
    
    LibPQ.fetch!(DataFrame, res)
end

In [ ]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    res = LibPQ.execute(conn, """
        WITH a AS (
            SELECT DISTINCT
                "userId",
                trunc(
                    cume_dist() OVER (
                        ORDER BY "userId"
                    ) * (\$1 -  0.00001)
                )::int4 + 1 AS chunk
            FROM ml_small
            ORDER BY "userId"
        )
        
        SELECT
            array_agg("userId")::int8[] as ids,
            chunk 
        FROM a
        GROUP BY chunk
        ORDER BY chunk;
    """, [10]);
    
    LibPQ.fetch!(DataFrame, res)
end

In [ ]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    res = LibPQ.execute(conn, """
        WITH a AS (
            SELECT DISTINCT
                "userId",
                trunc(
                    cume_dist() OVER (
                        ORDER BY "userId"
                    ) * (\$1 -  0.00001)
                )::int4 + 1 AS chunk
            FROM ml_small
            ORDER BY "userId"
        )
        
        SELECT
            array_agg("userId")::int8[] as ids,
            chunk 
        FROM a
        GROUP BY chunk
        ORDER BY chunk;
    """, [10]);
    
    rating_users = LibPQ.fetch!(DataFrame, res)[:ids][1]
    
    nt = fetch!(NamedTuple, LibPQ.execute(conn, """
        SELECT
            *
        FROM ml_small
        WHERE "userId" = ANY(\$1)
        ORDER BY "userId";
    """, [string("{", join(rating_users, ", "), "}")]));
end

In [28]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    collect(skipmissing(fetch!(NamedTuple, LibPQ.execute(conn, """
            WITH a AS (
                SELECT DISTINCT
                    "userId",
                    trunc(
                        cume_dist() OVER (
                            ORDER BY "userId"
                        ) * (\$1 -  0.00001)
                    )::int4 + 1 AS chunk
                FROM ml_small
                ORDER BY "userId"
            )

            SELECT
                array_agg("userId")::int8[] AS ids,
                chunk 
            FROM a
            GROUP BY chunk
            ORDER BY chunk;
        """, [4]))[:ids]));
end

  2.082460 seconds (61.18 k allocations: 3.352 MiB)


4-element Array{Union{Missing, AbstractArray{Int64,N} where N},1}:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  167, 168, 169, 170, 171, 172, 173, 174, 175, 176]                   
 [177, 178, 179, 180, 181, 182, 183, 184, 185, 186  …  315, 316, 317, 318, 319, 320, 321, 322, 323, 324]
 [325, 326, 327, 328, 329, 330, 331, 332, 333, 334  …  467, 468, 469, 470, 471, 472, 473, 474, 475, 476]
 [477, 478, 479, 480, 481, 482, 483, 484, 485, 486  …  601, 602, 603, 604, 605, 606, 607, 608, 609, 610]

In [29]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    collect(skipmissing(fetch!(NamedTuple, LibPQ.execute(conn, """
            WITH a AS (
                SELECT DISTINCT
                    "userId",
                    trunc(
                        cume_dist() OVER (
                            ORDER BY "userId"
                        ) * (\$1 -  0.00001)
                    )::int4 + 1 AS chunk
                FROM ml_small
                ORDER BY "userId"
            ),
            
            b AS (
                SELECT
                    chunk,
                    count(*) AS count
                FROM a
                GROUP BY "chunk"
            )
            
            SELECT
                stddev_pop(count)::float8
            FROM b
        """, [4]))));
end

  1.629217 seconds (54.64 k allocations: 3.060 MiB)


1-element Array{Array{Union{Missing, Float64},1},1}:
 [15.1245]

In [19]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    collect(skipmissing(fetch!(NamedTuple, LibPQ.execute(conn, """
            WITH a AS (
                SELECT DISTINCT
                    "userId",
                    trunc(
                        percent_rank() OVER (
                            ORDER BY "userId"
                        ) * \$1
                    )::int4 AS chunk
                FROM ml_small
                ORDER BY "userId"
            )

            SELECT
                array_agg("userId")::int8[] AS ids,
                chunk 
            FROM a
            GROUP BY chunk
            ORDER BY chunk;
        """, [4]))[:ids]));
end

  0.510244 seconds (61.18 k allocations: 3.353 MiB)


4-element Array{Union{Missing, AbstractArray{Int64,N} where N},1}:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  168, 169, 170, 171, 172, 173, 174, 175, 176, 177]                   
 [178, 179, 180, 181, 182, 183, 184, 185, 186, 187  …  316, 317, 318, 319, 320, 321, 322, 323, 324, 325]
 [326, 327, 328, 329, 330, 331, 332, 333, 334, 335  …  468, 469, 470, 471, 472, 473, 474, 475, 476, 477]
 [478, 479, 480, 481, 482, 483, 484, 485, 486, 487  …  601, 602, 603, 604, 605, 606, 607, 608, 609, 610]

In [20]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    collect(skipmissing(fetch!(NamedTuple, LibPQ.execute(conn, """
            WITH a AS (
                SELECT
                    "userId",
                    trunc(
                        percent_rank() OVER (
                            ORDER BY "userId"
                        ) * \$1
                    )::int4 AS chunk
                FROM ml_small
                ORDER BY "userId"
            ),
            
            b AS (
                SELECT
                    chunk,
                    count(*) AS count
                FROM a
                GROUP BY "chunk"
            )
            
            SELECT
                stddev_pop(count)::float8
            FROM b
        """, [4]))));
end

  0.530167 seconds (54.64 k allocations: 3.060 MiB)


1-element Array{Array{Union{Missing, Float64},1},1}:
 [481.017]

In [ ]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    fetch!(DataFrame, LibPQ.execute(conn, """
            SELECT
                array_agg("movieId") as movieid,
                "userId",
                array_agg("rating") as ratings
            FROM ml_small
            GROUP BY "userId"
            HAVING "userId" BETWEEN 1 AND 5;
        """));
end

In [22]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    fetch!(DataFrame, LibPQ.execute(conn, """
            SELECT
                array_agg("userId") as userid,
                "movieId",
                array_agg("rating") as ratings
            FROM ml_small
            GROUP BY "movieId"
            HAVING "movieId" BETWEEN 1 AND 10000;
        """));
end

  0.568934 seconds (2.17 M allocations: 86.521 MiB, 5.74% gc time)


,userid,movieId,ratings
,Union…⍰,Int64⍰,Union…⍰
1,"[1, 5, 7, 15, 17, 18, 19, 21, 27, 31, 32, 33, 40, 43, 44, 45, 46, 50, 54, 57, 63, 64, 66, 68, 71, 73, 76, 78, 82, 86, 89, 90, 91, 93, 96, 98, 103, 107, 112, 119, 121, 124, 130, 132, 134, 135, 137, 140, 141, 144, 145, 151, 153, 155, 156, 159, 160, 161, 166, 167, 169, 171, 177, 178, 179, 182, 185, 186, 191, 193, 200, 201, 202, 206, 213, 214, 216, 217, 219, 220, 223, 226, 229, 232, 233, 234, 239, 240, 247, 249, 252, 254, 263, 264, 266, 269, 270, 273, 274, 275, 276, 277, 279, 280, 282, 283, 288, 290, 291, 292, 293, 298, 304, 307, 314, 322, 323, 328, 330, 332, 334, 336, 337, 339, 341, 347, 350, 353, 357, 359, 364, 367, 372, 373, 378, 380, 381, 382, 385, 389, 391, 396, 399, 401, 411, 412, 414, 420, 422, 432, 434, 436, 438, 443, 448, 451, 453, 456, 460, 462, 468, 469, 470, 471, 474, 476, 477, 480, 483, 484, 488, 490, 492, 500, 504, 509, 514, 517, 522, 524, 525, 528, 529, 533, 534, 541, 544, 550, 555, 559, 560, 561, 562, 567, 570, 572, 573, 579, 580, 584, 587, 590, 596, 597, 599, 600, 601, 603, 604, 605, 606, 607, 608, 609, 610]",1,"[4.0, 4.0, 4.5, 2.5, 4.5, 3.5, 4.0, 3.5, 3.0, 5.0, 3.0, 3.0, 5.0, 5.0, 3.0, 4.0, 5.0, 3.0, 3.0, 5.0, 5.0, 4.0, 4.0, 2.5, 5.0, 4.5, 0.5, 4.0, 2.5, 4.0, 3.0, 3.0, 4.0, 3.0, 5.0, 4.5, 4.0, 4.0, 3.0, 3.5, 4.0, 4.0, 3.0, 2.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.5, 5.0, 5.0, 2.0, 3.0, 4.0, 4.5, 4.0, 4.0, 5.0, 3.5, 4.5, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 2.0, 3.5, 5.0, 4.0, 5.0, 3.5, 3.0, 3.0, 4.0, 3.5, 5.0, 3.5, 3.5, 5.0, 3.5, 3.0, 5.0, 4.0, 5.0, 5.0, 4.0, 4.5, 4.5, 4.0, 4.0, 2.0, 5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 4.0, 3.0, 4.5, 4.5, 3.0, 4.5, 4.0, 4.0, 4.0, 3.0, 2.0, 5.0, 4.0, 3.0, 3.5, 3.5, 5.0, 4.0, 4.0, 3.5, 4.0, 4.0, 4.0, 5.0, 5.0, 4.0, 5.0, 5.0, 4.0, 5.0, 5.0, 3.0, 3.0, 4.5, 5.0, 3.5, 4.5, 4.0, 5.0, 3.0, 5.0, 4.0, 3.5, 5.0, 2.0, 4.0, 4.0, 4.0, 2.5, 4.0, 4.0, 4.5, 4.0, 5.0, 5.0, 5.0, 5.0, 4.5, 1.5, 4.0, 4.0, 4.0, 5.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.5, 4.5, 3.5, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 2.5, 3.0, 5.0, 4.0, 3.0, 3.0, 4.0, 4.0, 5.0, 3.0, 4.0, 4.5, 3.5, 4.0, 4.0, 5.0, 4.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0, 3.0, 2.5, 4.0, 4.0, 3.0, 4.0, 2.5, 4.0, 2.5, 3.0, 5.0]"
2,"[6, 8, 18, 19, 20, 21, 27, 51, 62, 68, 82, 91, 93, 94, 103, 104, 107, 112, 117, 122, 125, 135, 140, 144, 149, 153, 160, 169, 177, 186, 202, 217, 219, 221, 222, 226, 230, 232, 240, 249, 259, 274, 276, 284, 288, 294, 298, 299, 304, 305, 307, 308, 318, 321, 322, 323, 330, 347, 353, 357, 359, 373, 380, 381, 395, 411, 414, 425, 426, 432, 434, 436, 446, 447, 448, 458, 470, 474, 475, 476, 477, 480, 482, 483, 484, 489, 497, 501, 512, 517, 523, 525, 527, 534, 559, 561, 563, 566, 570, 573, 586, 590, 592, 594, 599, 600, 602, 604, 605, 608]",2,"[4.0, 4.0, 3.0, 3.0, 3.0, 3.5, 4.0, 4.5, 4.0, 2.5, 3.0, 3.0, 5.0, 4.0, 4.0, 3.0, 5.0, 1.5, 3.0, 4.0, 4.0, 3.0, 3.5, 3.0, 1.0, 2.0, 4.0, 4.0, 3.5, 4.0, 4.0, 2.0, 2.5, 3.5, 2.5, 3.0, 2.5, 4.0, 5.0, 4.0, 2.0, 3.5, 4.0, 4.0, 2.0, 3.0, 0.5, 3.0, 4.0, 3.5, 2.5, 3.0, 3.5, 5.0, 3.0, 4.0, 1.5, 3.0, 4.0, 3.0, 3.5, 3.0, 5.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.5, 4.0, 2.5, 4.0, 3.0, 5.0, 3.0, 4.0, 3.0, 3.0, 4.5, 4.0, 4.0, 3.0, 4.5, 4.0, 2.5, 2.5, 2.5, 3.0, 3.0, 3.0, 4.5, 3.5, 4.0, 4.5, 4.0, 4.0, 2.5, 4.0, 3.5, 4.5, 4.0, 2.5, 4.0, 4.0, 2.5, 4.0, 4.0, 5.0, 3.5, 2.0]"
3,"[1, 6, 19, 32, 42, 43, 44, 51, 58, 64, 68, 91, 100, 102, 116, 117, 150, 151, 169, 179, 217, 226, 240, 269, 270, 288, 289, 294, 302, 307, 308, 321, 330, 337, 368, 410, 414, 448, 456, 470, 477, 480, 492, 501, 544, 552, 555, 588, 590, 594, 599, 608]",3,"[4.0, 5.0, 3.0, 3.0, 4.0, 5.0, 3.0, 4.0, 3.0, 3.5, 2.0, 3.0, 3.5, 5.0, 3.5, 3.0, 3.0, 3.0, 5.0, 4.0, 1.0, 3.5, 4.0, 4.0, 3.0, 4.0, 2.5, 1.0, 3.0, 3.5, 0.5, 3.0, 3.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 2.5, 4.0, 5.0, 3.0, 1.0, 5.0, 3.0, 3.0, 4.0, 1.5, 2.0]"
4,"[6, 14, 84, 162, 262, 411, 600]",4,"[3.0, 3.0, 3.0, 3.0, 1.0, 2.0, 1.5]"
5,"[6, 31, 43, 45, 58, 66, 68, 84, 103, 107, 111, 117, 120, 121, 147, 150, 169, 170, 181, 200, 229, 269, 270, 276, 288, 321, 337, 353, 389, 402, 

In [21]:
@time LibPQ.Connection("host=localhost dbname=postgres"; type_map=Dict(:int4=>Int64, :float4=>Float64)) do conn
    f(i::Int64) = fetch!(DataFrame, LibPQ.execute(conn, """
        SELECT
            array_agg("userId") as userid,
            "movieId",
            array_agg("rating") as ratings
        FROM ml_small
        GROUP BY "movieId"
        HAVING "movieId" = \$1;
    """, [i]));
    
    res = asyncmap(f, 1:10000, ntasks=100)
end

  7.720371 seconds (7.09 M allocations: 379.134 MiB, 3.18% gc time)


10000-element Array{DataFrame,1}:
 1×3 DataFrame. Omitted printing of 3 columns
│ Row │ │     │ ├─────┼
│ 1   │                                                                                                                                                                                                                        
 1×3 DataFrame. Omitted printing of 3 columns
│ Row │ │     │ ├─────┼
│ 1   │                                                                                                                                                                                                                        
 1×3 DataFrame. Omitted printing of 3 columns
│ Row │ │     │ ├─────┼
│ 1   │                                                                                                                                                                                                                        
 1×3 DataFrame. Omitted printing of 1 columns
│ Row │ userid                        